In [2]:
!pip install neo4j neo4j-graphrag -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [3]:
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.retrievers import VectorCypherRetriever
from neo4j_graphrag.generation import GraphRAG

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [4]:
def load_properties(path):
    data = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.split("=", 1)
                data[k.strip()] = v.strip()
    return data

In [5]:
import os
props = load_properties("openai_key.txt")

os.environ["OPENAI_KEY"] = props["OPENAI_KEY"]
os.environ["NEO4J_URI"] = props["NEO4J_URI"]
os.environ["NEO4J_USERNAME"] = props["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"] = props["NEO4J_PASSWORD"]
#os.environ["NEO4J_DATABASE"] = props["NEO4J_DATABASE"]


In [6]:
neo4j_driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

In [7]:
neo4j_driver.verify_connectivity()

In [8]:
llm = OpenAILLM(
    model_name="gpt-4o",
    api_key=os.environ["OPENAI_KEY"],
    model_params={
        "temperature": 0,
        "response_format": {"type": "json_object"},
    }
)

In [9]:
embedder = OpenAIEmbeddings(
    api_key=os.environ["OPENAI_KEY"],
    model="text-embedding-ada-002"
    )

In [10]:
#without text splitter
'''
kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=neo4j_driver,
    neo4j_database=os.getenv("NEO4J_DATABASE"),
    embedder=embedder,
    from_pdf=True,
)

SyntaxError: incomplete input (ipython-input-3036860817.py, line 2)

In [ ]:
#using text splitter for creating data chunks
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter

text_splitter = FixedSizeSplitter(chunk_size=500, chunk_overlap=50)

In [ ]:
#Define schema - what nodes types and relationships I am interested to extract
NODE_TYPES = [
    "Technology",
    "Concept",
    "Example",
    "Process",
    "Challenge",
    {"label": "Benefit", "description": "A benefit or advantage of using a technology or approach."},
    {
        "label": "Resource",
        "description": "A related learning resource such as a book, article, video, or course.",
        "properties": [
            {"name": "name", "type": "STRING", "required": True},
            {"name": "type", "type": "STRING"}
        ]
    },
]

RELATIONSHIP_TYPES = [
    "RELATED_TO",
    "PART_OF",
    "USED_IN",
    "LEADS_TO",
    "HAS_CHALLENGE",
    "LEADS_TO",
    "CITES"
]

PATTERNS = [
    ("Technology", "RELATED_TO", "Technology"),
    ("Concept", "RELATED_TO", "Technology"),
    ("Example", "USED_IN", "Technology"),
    ("Process", "PART_OF", "Technology"),
    ("Technology", "HAS_CHALLENGE", "Challenge"),
    ("Concept", "HAS_CHALLENGE", "Challenge"),
    ("Technology", "LEADS_TO", "Benefit"),
    ("Process", "LEADS_TO", "Benefit"),
    ("Resource", "CITES", "Technology"),
]


In [ ]:
kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=neo4j_driver,
    neo4j_database=os.getenv("NEO4J_DATABASE"),
    embedder=embedder,
    from_pdf=True,
    text_splitter=text_splitter,
     schema={
        "node_types": NODE_TYPES,
        "relationship_types": RELATIONSHIP_TYPES,
        "patterns": PATTERNS
    },
)

In [ ]:
pdf_file = "genai-fundamentals_4-integrating-neo4j_6-frameworks.pdf"
result = await kg_builder.run_async(file_path=pdf_file)
print(result)

##The unstructured part of the graph is known as the **Lexical Graph**
##The structured part is known as the **Domain Graph**

In [ ]:
#Loading structured data to the knowledge Graph

In [ ]:
import csv

csv_file = "StructuredData/docs.csv"

#docs_csv = csv.DictReader(
#    open(os.path.join(data_path, "docs.csv"), encoding="utf8", newline='')
#)

# Open the CSV file
with open(csv_file, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)  # Reads each row as a dictionary
    for row in reader:
        print(row)  # Each row is a dict: {'filename': 'doc1.pdf', 'title': 'Intro', ...}
        # Access columns
        print(row['filename'], row['course'], row['module'],row['lesson'],row['url'])
        pdf_path=row['filename']
        url=row['url']
        lesson=row['lesson']
        course=row['course']
        module=row['module']




In [ ]:
cypher = """
MATCH (d:Document {path: $pdf_path})
MERGE (l:Lesson {url: $url})
SET l.name = $lesson,
    l.module = $module,
    l.course = $course
MERGE (d)-[:PDF_OF]->(l)
"""


In [ ]:
 # Create structured graph

params = {
    "pdf_path": pdf_path,
    "url": url,
    "lesson": lesson,
    "module":module ,
    "course": course
}

records, summary, keys = neo4j_driver.execute_query(
    cypher,
    parameters_=params,
    database_=os.getenv("NEO4J_DATABASE")
)

# For fast retrieval creating Vector Index

In [11]:
with neo4j_driver.session() as session:
    session.run("""
    CREATE VECTOR INDEX chunkEmbedding IF NOT EXISTS
    FOR (n:Chunk)
    ON n.embedding
    OPTIONS {
      indexConfig: {
        `vector.dimensions`: 1536,
        `vector.similarity_function`: 'cosine'
      }
    };
    """)


##Creating retriever using vector Index

In [12]:
embedder = OpenAIEmbeddings(
    api_key=os.environ["OPENAI_KEY"],
    model="text-embedding-ada-002"
    )

In [13]:
# Define retrieval query
retrieval_query = """
RETURN node.text as text, score
"""

In [14]:
# Create retriever
retriever = VectorCypherRetriever(
    neo4j_driver,
    neo4j_database=os.getenv("NEO4J_DATABASE"),
    index_name="chunkEmbedding",
    embedder=embedder,
    retrieval_query=retrieval_query,
)

In [15]:
llm_4_retriever = OpenAILLM(
    model_name="gpt-4o",
    api_key=os.environ["OPENAI_KEY"]

)

In [16]:
# Create GraphRAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm_4_retriever)

In [23]:
# Search
import pprint

query_text = "find details for spring ecosystem project with document link"

response = rag.search(
    query_text=query_text,
    retriever_config={"top_k": 5},
    return_context=True
)

#print(response.answer)
pprint.pprint((response.answer), width=50)

('The Spring ecosystem project for integrating '
 'AI capabilities into Java applications, '
 'including support for Neo4j, is referred to as '
 '"Spring AI." For more details, you can visit '
 'the project documentation at the following '
 'link: [Spring AI '
 'Documentation](https://spring.io/projects/spring-ai).')


In [25]:
pprint.pprint(("CONTEXT:", response.retriever_result.items))

('CONTEXT:',
 [RetrieverResultItem(content="<Record text='- A Spring\\necosystem project for integrating AI capabilities into Java applications,\\nincluding Neo4j support.\\n* link:https://docs.langchain4j.dev/[Langchain4j^] - A Java version of\\nLangChain, supporting Neo4j integration for LLM and RAG workflows.\\n* link:https://haystack.deepset.ai/[Haystack^] - An open-source framework\\nfor building search and question-answering systems, with Neo4j integration\\nfor graph-based retrieval.\\n*\\nlink:https://learn.microsoft.com/en-us/semantic-kernel/overview/[Semantic\\n' score=0.9166557788848877>", metadata=None),
  RetrieverResultItem(content="<Record text='support for Neo4j as a vector store and knowledge graph.\\n* link:https://www.langchain.com/[LangChainJS^] - The\\nJavaScript/TypeScript version of LangChain, enabling GenAI workflows in\\nNode.js and browser environments.\\n* link:https://www.llamaindex.ai/[LlamaIndex^] -  A data framework for\\nconnecting LLMs to external data,